# 통합 생명과학 연구 에이전트 - Amazon Bedrock AgentCore

이 노트북에서는 모든 노트북(01~04)의 도구들을 통합한 종합적인 생명과학 연구 에이전트를 Amazon Bedrock AgentCore Runtime으로 배포합니다.

## 통합되는 기능
- **외부 데이터베이스** (01_external_dbs): Arxiv, ChEMBL, PubMed, ClinicalTrials
- **내부 데이터베이스** (02_internal_dbs): PostgreSQL 임상/유전체 데이터
- **하이브리드 도구** (03_hybrid_tools): Knowledge Base, Redshift 등
- **단백질 설계** (04_protein_design): AWS HealthOmics 워크플로우

## 학습 목표
- 모든 도구를 통합한 종합 Agent 구현
- AgentCore Runtime에 배포 및 운영
- 다양한 생명과학 연구 시나리오 테스트

## 1. 환경 설정

In [ ]:
# 필요한 패키지 설치
%pip install bedrock-agentcore-starter-toolkit strands-agents strands-agents-tools boto3 \
    mcp arxiv chembl-webresource-client python-dateutil pubmedmcp \
    psycopg2-binary --quiet

In [1]:
# 라이브러리 임포트
import sys
import json
import logging
from typing import Dict, Any

# AWS SDK
import boto3
from boto3.session import Session

# Bedrock AgentCore
from bedrock_agentcore_starter_toolkit import Runtime
from bedrock_agentcore.runtime import BedrockAgentCoreApp

# Strands Agents
from strands import Agent, tool
from strands.models import BedrockModel
from strands_tools import calculator

# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [2]:
%%writefile integrated_research_agent.py
"""
통합 생명과학 연구 에이전트 for Amazon Bedrock AgentCore
모든 도구 통합: 외부DB, 내부DB, 하이브리드, 단백질 설계
"""
import json
import logging
import os
from typing import Dict, Any, List, Optional
from collections import defaultdict
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands import Agent, tool
from strands.models import BedrockModel

# 외부 라이브러리
import arxiv
from chembl_webresource_client.new_client import new_client as chembl_client
import httpx
from defusedxml import ElementTree as ET
import psycopg2
import boto3

# .env 파일에서 환경 변수 로드
try:
    from dotenv import load_dotenv
    load_dotenv()
except ImportError:
    # dotenv가 없으면 환경 변수에서 직접 읽기
    pass

app = BedrockAgentCoreApp()
logger = logging.getLogger(__name__)

# AWS 클라이언트
omics_client = boto3.client('omics')
s3_client = boto3.client('s3')
bedrock_client = boto3.client('bedrock-runtime')
knowledge_base_client = boto3.client('bedrock-agent-runtime')

# =========================
# 1. 외부 데이터베이스 도구
# =========================

@tool
def search_arxiv(query: str, max_results: int = 5) -> List[Dict[str, Any]]:
    """Arxiv에서 학술 논문을 검색합니다
    
    Args:
        query: 검색 쿼리
        max_results: 최대 결과 수 (기본값: 5)
    
    Returns:
        논문 정보 리스트
    """
    try:
        client = arxiv.Client()
        search = arxiv.Search(
            query=query,
            max_results=max_results,
            sort_by=arxiv.SortCriterion.SubmittedDate
        )
        
        results = []
        for paper in client.results(search):
            results.append({
                "title": paper.title,
                "authors": [author.name for author in paper.authors],
                "abstract": paper.summary[:500],
                "url": paper.pdf_url,
                "published": paper.published.isoformat()
            })
        return results
    except Exception as e:
        logger.error(f"Arxiv 검색 오류: {e}")
        return [{"error": str(e)}]

@tool
def search_compound(compound_name: str) -> Dict[str, Any]:
    """ChEMBL에서 화합물 정보를 검색합니다
    
    Args:
        compound_name: 화합물명
    
    Returns:
        화합물 정보 및 활성 데이터
    """
    try:
        molecule = chembl_client.molecule.filter(
            pref_name__iexact=compound_name
        ).only(['molecule_chembl_id', 'pref_name', 'max_phase'])
        
        if molecule:
            mol_data = molecule[0]
            # IC50 활성 데이터
            activity = chembl_client.activity.filter(
                molecule_chembl_id=mol_data['molecule_chembl_id']
            ).filter(standard_type="IC50").only(
                ['pchembl_value', 'assay_description', 'canonical_smiles']
            )[:10]
            
            return {
                "chembl_id": mol_data['molecule_chembl_id'],
                "name": mol_data['pref_name'],
                "max_phase": mol_data.get('max_phase'),
                "activities": list(activity)
            }
        return {"error": "Compound not found"}
    except Exception as e:
        logger.error(f"ChEMBL 검색 오류: {e}")
        return {"error": str(e)}

@tool
def search_pubmed(query: str, max_results: int = 5) -> List[Dict[str, Any]]:
    """PubMed에서 의학 문헌을 검색합니다
    
    Args:
        query: 검색 쿼리
        max_results: 최대 결과 수 (기본값: 5)
    
    Returns:
        논문 정보 리스트
    """
    try:
        base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils"
        
        # 검색
        search_params = {
            "db": "pubmed",
            "term": query,
            "retmax": max_results,
            "retmode": "json"
        }
        
        search_response = httpx.get(
            f"{base_url}/esearch.fcgi",
            params=search_params
        )
        search_data = search_response.json()
        
        id_list = search_data["esearchresult"].get("idlist", [])
        if not id_list:
            return []
        
        # 상세 정보
        fetch_params = {
            "db": "pubmed",
            "id": ",".join(id_list),
            "retmode": "xml"
        }
        
        fetch_response = httpx.get(
            f"{base_url}/efetch.fcgi",
            params=fetch_params
        )
        
        # XML 파싱
        root = ET.fromstring(fetch_response.text)
        articles = []
        
        for article in root.findall(".//PubmedArticle"):
            title = article.find(".//ArticleTitle")
            abstract = article.find(".//AbstractText")
            pmid = article.find(".//PMID")
            year = article.find(".//PubDate/Year")
            
            articles.append({
                "pmid": pmid.text if pmid is not None else "",
                "title": title.text if title is not None else "",
                "abstract": abstract.text[:500] if abstract is not None else "",
                "year": year.text if year is not None else ""
            })
        
        return articles
    except Exception as e:
        logger.error(f"PubMed 검색 오류: {e}")
        return [{"error": str(e)}]

# =========================
# 2. 내부 데이터베이스 도구 (PostgreSQL)
# =========================

# 데이터베이스 설정 - 환경변수에서 가져오기
DB_CONFIG = {
    "host": os.environ.get('DB_HOST', 'localhost'),
    "port": int(os.environ.get('DB_PORT', 5432)),
    "database": os.environ.get('DB_NAME', 'agentdb'),
    "user": os.environ.get('DB_USER', 'dbadmin'),
    "password": os.environ.get('DB_PASSWORD', 'postgres')
}

@tool
def query_clinical_database(query: str) -> Dict[str, Any]:
    """내부 PostgreSQL 임상 데이터베이스를 쿼리합니다
    
    Args:
        query: SQL 쿼리 또는 자연어 질문
    
    Returns:
        쿼리 결과
    """
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        with conn.cursor() as cursor:
            # SQL 쿼리인지 확인
            if query.strip().upper().startswith(('SELECT', 'WITH')):
                cursor.execute(query)
            else:
                # 자연어를 간단한 SQL로 변환 (실제로는 더 복잡한 NLP 필요)
                cursor.execute(
                    "SELECT table_name FROM information_schema.tables WHERE table_schema='public'"
                )
            
            results = cursor.fetchall()
            return {
                "status": "success",
                "data": results[:100],  # 결과 제한
                "count": len(results)
            }
    except Exception as e:
        logger.error(f"Database 쿼리 오류: {e}")
        return {"error": str(e)}
    finally:
        if 'conn' in locals():
            conn.close()

@tool
def get_database_schema() -> Dict[str, Any]:
    """데이터베이스 스키마 정보를 가져옵니다
    
    Returns:
        테이블 및 컬럼 정보
    """
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        with conn.cursor() as cursor:
            cursor.execute("""
                SELECT
                    c.relname AS table_name,
                    a.attname AS column_name,
                    pg_catalog.format_type(a.atttypid, a.atttypmod) AS data_type
                FROM pg_catalog.pg_attribute a
                JOIN pg_catalog.pg_class c ON a.attrelid = c.oid
                JOIN pg_catalog.pg_namespace n ON c.relnamespace = n.oid
                WHERE a.attnum > 0 AND NOT a.attisdropped
                    AND n.nspname = 'public'
                    AND c.relkind = 'r'
                ORDER BY c.relname, a.attnum;
            """)
            
            rows = cursor.fetchall()
            tables = defaultdict(list)
            for table_name, column_name, data_type in rows:
                tables[table_name].append({
                    "column": column_name,
                    "type": data_type
                })
            
            return {
                "status": "success",
                "schema": dict(tables)
            }
    except Exception as e:
        logger.error(f"스키마 조회 오류: {e}")
        return {"error": str(e)}
    finally:
        if 'conn' in locals():
            conn.close()

# =========================
# 3. 단백질 설계 도구
# =========================

@tool
def trigger_protein_optimization(
    sequence: str,
    parallel_chains: int = 10,
    steps_per_chain: int = 100,
    max_mutations: int = 15
) -> Dict[str, Any]:
    """단백질 서열 최적화 워크플로우를 시작합니다
    
    Args:
        sequence: 단백질 서열 (아미노산)
        parallel_chains: 병렬 체인 수
        steps_per_chain: 체인당 단계 수
        max_mutations: 최대 변이 수
    
    Returns:
        워크플로우 실행 정보
    """
    try:
        # 서열 검증
        valid_amino_acids = set('ACDEFGHIKLMNPQRSTVWY')
        if not all(aa in valid_amino_acids for aa in sequence.upper()):
            return {"error": "Invalid amino acid sequence"}
        
        # 워크플로우 파라미터
        workflow_id = os.environ.get('WORKFLOW_ID', '8600464')
        role_arn = os.environ.get('WORKFLOW_ROLE_ARN')
        s3_bucket = os.environ.get('S3_BUCKET')
        
        # HealthOmics 워크플로우 실행
        response = omics_client.start_run(
            workflowId=workflow_id,
            roleArn=role_arn,
            outputUri=f"s3://{s3_bucket}/outputs",
            parameters={
                "sequence": sequence,
                "parallel_chains": str(parallel_chains),
                "steps_per_chain": str(steps_per_chain),
                "max_mutations": str(max_mutations)
            }
        )
        
        return {
            "status": "started",
            "run_id": response['id'],
            "arn": response['arn']
        }
    except Exception as e:
        logger.error(f"단백질 최적화 시작 오류: {e}")
        return {"error": str(e)}

@tool
def monitor_protein_workflow(run_id: str) -> Dict[str, Any]:
    """단백질 최적화 워크플로우 상태를 확인합니다
    
    Args:
        run_id: 워크플로우 실행 ID
    
    Returns:
        워크플로우 상태 및 결과
    """
    try:
        response = omics_client.get_run(id=run_id)
        
        status = response['status']
        result = {
            "status": status,
            "run_id": run_id
        }
        
        if status == "COMPLETED":
            # 결과 파일 가져오기
            output_uri = response.get('outputUri')
            if output_uri:
                result["output_location"] = output_uri
                # S3에서 결과 읽기 (간단한 예시)
                bucket = output_uri.split('/')[2]
                key = '/'.join(output_uri.split('/')[3:])
                try:
                    obj = s3_client.get_object(Bucket=bucket, Key=key)
                    result["optimized_sequence"] = obj['Body'].read().decode('utf-8')
                except:
                    pass
        
        return result
    except Exception as e:
        logger.error(f"워크플로우 모니터링 오류: {e}")
        return {"error": str(e)}

# =========================
# 4. Knowledge Base 도구
# =========================

@tool
def query_knowledge_base(query: str, kb_id: Optional[str] = None) -> Dict[str, Any]:
    """Amazon Bedrock Knowledge Base를 쿼리합니다
    
    Args:
        query: 검색 쿼리
        kb_id: Knowledge Base ID (선택)
    
    Returns:
        검색 결과
    """
    try:
        kb_id = kb_id or os.environ.get('KNOWLEDGE_BASE_ID')
        if not kb_id:
            return {"error": "Knowledge Base ID not configured"}
        
        response = knowledge_base_client.retrieve(
            knowledgeBaseId=kb_id,
            retrievalQuery={'text': query},
            retrievalConfiguration={
                'vectorSearchConfiguration': {
                    'numberOfResults': 5
                }
            }
        )
        
        results = []
        for item in response.get('retrievalResults', []):
            results.append({
                "content": item['content']['text'],
                "score": item.get('score', 0)
            })
        
        return {
            "status": "success",
            "results": results
        }
    except Exception as e:
        logger.error(f"Knowledge Base 쿼리 오류: {e}")
        return {"error": str(e)}

# =========================
# 메인 Agent 설정
# =========================

# 모델 설정
model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"
model = BedrockModel(model_id=model_id)

# 통합 Agent 생성
agent = Agent(
    model=model,
    tools=[
        # 외부 데이터베이스
        search_arxiv,
        search_compound,
        search_pubmed,
        # 내부 데이터베이스
        query_clinical_database,
        get_database_schema,
        # 단백질 설계
        trigger_protein_optimization,
        monitor_protein_workflow,
        # Knowledge Base
        query_knowledge_base
    ],
    system_prompt="""당신은 종합적인 생명과학 연구 AI 어시스턴트입니다.
    
    사용 가능한 도구:
    1. **외부 데이터베이스**: Arxiv, ChEMBL, PubMed를 통한 문헌 및 화합물 검색
    2. **내부 데이터베이스**: PostgreSQL 임상/유전체 데이터 분석
    3. **단백질 설계**: AWS HealthOmics를 통한 단백질 최적화
    4. **Knowledge Base**: 조직 내부 지식 검색
    
    연구자의 질문에 대해:
    - 적절한 도구를 선택하여 사용
    - 여러 소스의 정보를 종합하여 포괄적인 답변 제공
    - 과학적 정확성 유지
    - 한국어로 친절하게 응답"""
)

@app.entrypoint
def integrated_research_agent(payload: Dict[str, Any]) -> str:
    """AgentCore 엔트리포인트"""
    user_input = payload.get("prompt", "")
    logger.info(f"Processing request: {user_input}")
    
    try:
        response = agent(user_input)
        return response.message['content'][0]['text']
    except Exception as e:
        logger.error(f"Error: {str(e)}")
        return f"오류가 발생했습니다: {str(e)}"

if __name__ == "__main__":
    app.run()

Overwriting integrated_research_agent.py


## 2. 통합 Agent 구현

모든 노트북의 도구들을 하나의 Agent에 통합합니다.

In [3]:
%%writefile integrated_research_agent.py
"""
통합 생명과학 연구 에이전트 for Amazon Bedrock AgentCore
모든 도구 통합: 외부DB, 내부DB, 하이브리드, 단백질 설계
"""
import json
import logging
import os
from typing import Dict, Any, List, Optional
from collections import defaultdict
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands import Agent, tool
from strands.models import BedrockModel

# 외부 라이브러리
import arxiv
from chembl_webresource_client.new_client import new_client as chembl_client
import httpx
from defusedxml import ElementTree as ET
import psycopg2
import boto3

app = BedrockAgentCoreApp()
logger = logging.getLogger(__name__)

# AWS 클라이언트
omics_client = boto3.client('omics')
s3_client = boto3.client('s3')
bedrock_client = boto3.client('bedrock-runtime')
knowledge_base_client = boto3.client('bedrock-agent-runtime')

# =========================
# 1. 외부 데이터베이스 도구
# =========================

@tool
def search_arxiv(query: str, max_results: int = 5) -> List[Dict[str, Any]]:
    """Arxiv에서 학술 논문을 검색합니다
    
    Args:
        query: 검색 쿼리
        max_results: 최대 결과 수 (기본값: 5)
    
    Returns:
        논문 정보 리스트
    """
    try:
        client = arxiv.Client()
        search = arxiv.Search(
            query=query,
            max_results=max_results,
            sort_by=arxiv.SortCriterion.SubmittedDate
        )
        
        results = []
        for paper in client.results(search):
            results.append({
                "title": paper.title,
                "authors": [author.name for author in paper.authors],
                "abstract": paper.summary[:500],
                "url": paper.pdf_url,
                "published": paper.published.isoformat()
            })
        return results
    except Exception as e:
        logger.error(f"Arxiv 검색 오류: {e}")
        return [{"error": str(e)}]

@tool
def search_compound(compound_name: str) -> Dict[str, Any]:
    """ChEMBL에서 화합물 정보를 검색합니다
    
    Args:
        compound_name: 화합물명
    
    Returns:
        화합물 정보 및 활성 데이터
    """
    try:
        molecule = chembl_client.molecule.filter(
            pref_name__iexact=compound_name
        ).only(['molecule_chembl_id', 'pref_name', 'max_phase'])
        
        if molecule:
            mol_data = molecule[0]
            # IC50 활성 데이터
            activity = chembl_client.activity.filter(
                molecule_chembl_id=mol_data['molecule_chembl_id']
            ).filter(standard_type="IC50").only(
                ['pchembl_value', 'assay_description', 'canonical_smiles']
            )[:10]
            
            return {
                "chembl_id": mol_data['molecule_chembl_id'],
                "name": mol_data['pref_name'],
                "max_phase": mol_data.get('max_phase'),
                "activities": list(activity)
            }
        return {"error": "Compound not found"}
    except Exception as e:
        logger.error(f"ChEMBL 검색 오류: {e}")
        return {"error": str(e)}

@tool
def search_pubmed(query: str, max_results: int = 5) -> List[Dict[str, Any]]:
    """PubMed에서 의학 문헌을 검색합니다
    
    Args:
        query: 검색 쿼리
        max_results: 최대 결과 수 (기본값: 5)
    
    Returns:
        논문 정보 리스트
    """
    try:
        base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils"
        
        # 검색
        search_params = {
            "db": "pubmed",
            "term": query,
            "retmax": max_results,
            "retmode": "json"
        }
        
        search_response = httpx.get(
            f"{base_url}/esearch.fcgi",
            params=search_params
        )
        search_data = search_response.json()
        
        id_list = search_data["esearchresult"].get("idlist", [])
        if not id_list:
            return []
        
        # 상세 정보
        fetch_params = {
            "db": "pubmed",
            "id": ",".join(id_list),
            "retmode": "xml"
        }
        
        fetch_response = httpx.get(
            f"{base_url}/efetch.fcgi",
            params=fetch_params
        )
        
        # XML 파싱
        root = ET.fromstring(fetch_response.text)
        articles = []
        
        for article in root.findall(".//PubmedArticle"):
            title = article.find(".//ArticleTitle")
            abstract = article.find(".//AbstractText")
            pmid = article.find(".//PMID")
            year = article.find(".//PubDate/Year")
            
            articles.append({
                "pmid": pmid.text if pmid is not None else "",
                "title": title.text if title is not None else "",
                "abstract": abstract.text[:500] if abstract is not None else "",
                "year": year.text if year is not None else ""
            })
        
        return articles
    except Exception as e:
        logger.error(f"PubMed 검색 오류: {e}")
        return [{"error": str(e)}]

# =========================
# 2. 내부 데이터베이스 도구 (PostgreSQL)
# =========================

# 데이터베이스 설정 - 환경변수에서 가져오기
DB_CONFIG = {
    "host": os.environ.get('DB_HOST', 'localhost'),
    "port": int(os.environ.get('DB_PORT', 5432)),
    "database": os.environ.get('DB_NAME', 'agentdb'),
    "user": os.environ.get('DB_USER', 'dbadmin'),
    "password": os.environ.get('DB_PASSWORD', 'postgres')
}

@tool
def query_clinical_database(query: str) -> Dict[str, Any]:
    """내부 PostgreSQL 임상 데이터베이스를 쿼리합니다
    
    Args:
        query: SQL 쿼리 또는 자연어 질문
    
    Returns:
        쿼리 결과
    """
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        with conn.cursor() as cursor:
            # SQL 쿼리인지 확인
            if query.strip().upper().startswith(('SELECT', 'WITH')):
                cursor.execute(query)
            else:
                # 자연어를 간단한 SQL로 변환 (실제로는 더 복잡한 NLP 필요)
                cursor.execute(
                    "SELECT table_name FROM information_schema.tables WHERE table_schema='public'"
                )
            
            results = cursor.fetchall()
            return {
                "status": "success",
                "data": results[:100],  # 결과 제한
                "count": len(results)
            }
    except Exception as e:
        logger.error(f"Database 쿼리 오류: {e}")
        return {"error": str(e)}
    finally:
        if 'conn' in locals():
            conn.close()

@tool
def get_database_schema() -> Dict[str, Any]:
    """데이터베이스 스키마 정보를 가져옵니다
    
    Returns:
        테이블 및 컬럼 정보
    """
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        with conn.cursor() as cursor:
            cursor.execute("""
                SELECT
                    c.relname AS table_name,
                    a.attname AS column_name,
                    pg_catalog.format_type(a.atttypid, a.atttypmod) AS data_type
                FROM pg_catalog.pg_attribute a
                JOIN pg_catalog.pg_class c ON a.attrelid = c.oid
                JOIN pg_catalog.pg_namespace n ON c.relnamespace = n.oid
                WHERE a.attnum > 0 AND NOT a.attisdropped
                    AND n.nspname = 'public'
                    AND c.relkind = 'r'
                ORDER BY c.relname, a.attnum;
            """)
            
            rows = cursor.fetchall()
            tables = defaultdict(list)
            for table_name, column_name, data_type in rows:
                tables[table_name].append({
                    "column": column_name,
                    "type": data_type
                })
            
            return {
                "status": "success",
                "schema": dict(tables)
            }
    except Exception as e:
        logger.error(f"스키마 조회 오류: {e}")
        return {"error": str(e)}
    finally:
        if 'conn' in locals():
            conn.close()

# =========================
# 3. 단백질 설계 도구
# =========================

@tool
def trigger_protein_optimization(
    sequence: str,
    parallel_chains: int = 10,
    steps_per_chain: int = 100,
    max_mutations: int = 15
) -> Dict[str, Any]:
    """단백질 서열 최적화 워크플로우를 시작합니다
    
    Args:
        sequence: 단백질 서열 (아미노산)
        parallel_chains: 병렬 체인 수
        steps_per_chain: 체인당 단계 수
        max_mutations: 최대 변이 수
    
    Returns:
        워크플로우 실행 정보
    """
    try:
        # 서열 검증
        valid_amino_acids = set('ACDEFGHIKLMNPQRSTVWY')
        if not all(aa in valid_amino_acids for aa in sequence.upper()):
            return {"error": "Invalid amino acid sequence"}
        
        # 워크플로우 파라미터
        workflow_id = os.environ.get('WORKFLOW_ID', '8600464')
        role_arn = os.environ.get('WORKFLOW_ROLE_ARN')
        s3_bucket = os.environ.get('S3_BUCKET')
        
        # HealthOmics 워크플로우 실행
        response = omics_client.start_run(
            workflowId=workflow_id,
            roleArn=role_arn,
            outputUri=f"s3://{s3_bucket}/outputs",
            parameters={
                "sequence": sequence,
                "parallel_chains": str(parallel_chains),
                "steps_per_chain": str(steps_per_chain),
                "max_mutations": str(max_mutations)
            }
        )
        
        return {
            "status": "started",
            "run_id": response['id'],
            "arn": response['arn']
        }
    except Exception as e:
        logger.error(f"단백질 최적화 시작 오류: {e}")
        return {"error": str(e)}

@tool
def monitor_protein_workflow(run_id: str) -> Dict[str, Any]:
    """단백질 최적화 워크플로우 상태를 확인합니다
    
    Args:
        run_id: 워크플로우 실행 ID
    
    Returns:
        워크플로우 상태 및 결과
    """
    try:
        response = omics_client.get_run(id=run_id)
        
        status = response['status']
        result = {
            "status": status,
            "run_id": run_id
        }
        
        if status == "COMPLETED":
            # 결과 파일 가져오기
            output_uri = response.get('outputUri')
            if output_uri:
                result["output_location"] = output_uri
                # S3에서 결과 읽기 (간단한 예시)
                bucket = output_uri.split('/')[2]
                key = '/'.join(output_uri.split('/')[3:])
                try:
                    obj = s3_client.get_object(Bucket=bucket, Key=key)
                    result["optimized_sequence"] = obj['Body'].read().decode('utf-8')
                except:
                    pass
        
        return result
    except Exception as e:
        logger.error(f"워크플로우 모니터링 오류: {e}")
        return {"error": str(e)}

# =========================
# 4. Knowledge Base 도구
# =========================

@tool
def query_knowledge_base(query: str, kb_id: Optional[str] = None) -> Dict[str, Any]:
    """Amazon Bedrock Knowledge Base를 쿼리합니다
    
    Args:
        query: 검색 쿼리
        kb_id: Knowledge Base ID (선택)
    
    Returns:
        검색 결과
    """
    try:
        kb_id = kb_id or os.environ.get('KNOWLEDGE_BASE_ID')
        if not kb_id:
            return {"error": "Knowledge Base ID not configured"}
        
        response = knowledge_base_client.retrieve(
            knowledgeBaseId=kb_id,
            retrievalQuery={'text': query},
            retrievalConfiguration={
                'vectorSearchConfiguration': {
                    'numberOfResults': 5
                }
            }
        )
        
        results = []
        for item in response.get('retrievalResults', []):
            results.append({
                "content": item['content']['text'],
                "score": item.get('score', 0)
            })
        
        return {
            "status": "success",
            "results": results
        }
    except Exception as e:
        logger.error(f"Knowledge Base 쿼리 오류: {e}")
        return {"error": str(e)}

# =========================
# 메인 Agent 설정
# =========================

# 모델 설정
model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"
model = BedrockModel(model_id=model_id)

# 통합 Agent 생성
agent = Agent(
    model=model,
    tools=[
        # 외부 데이터베이스
        search_arxiv,
        search_compound,
        search_pubmed,
        # 내부 데이터베이스
        query_clinical_database,
        get_database_schema,
        # 단백질 설계
        trigger_protein_optimization,
        monitor_protein_workflow,
        # Knowledge Base
        query_knowledge_base
    ],
    system_prompt="""당신은 종합적인 생명과학 연구 AI 어시스턴트입니다.
    
    사용 가능한 도구:
    1. **외부 데이터베이스**: Arxiv, ChEMBL, PubMed를 통한 문헌 및 화합물 검색
    2. **내부 데이터베이스**: PostgreSQL 임상/유전체 데이터 분석
    3. **단백질 설계**: AWS HealthOmics를 통한 단백질 최적화
    4. **Knowledge Base**: 조직 내부 지식 검색
    
    연구자의 질문에 대해:
    - 적절한 도구를 선택하여 사용
    - 여러 소스의 정보를 종합하여 포괄적인 답변 제공
    - 과학적 정확성 유지
    - 한국어로 친절하게 응답"""
)

@app.entrypoint
def integrated_research_agent(payload: Dict[str, Any]) -> str:
    """AgentCore 엔트리포인트"""
    user_input = payload.get("prompt", "")
    logger.info(f"Processing request: {user_input}")
    
    try:
        response = agent(user_input)
        return response.message['content'][0]['text']
    except Exception as e:
        logger.error(f"Error: {str(e)}")
        return f"오류가 발생했습니다: {str(e)}"

if __name__ == "__main__":
    app.run()

Overwriting integrated_research_agent.py


## 3. Requirements 파일 생성

In [4]:
%%writefile requirements.txt
strands-agents>=0.5.0
strands-agents-tools>=0.1.0
boto3>=1.34.0
mcp>=0.1.0
arxiv>=2.1.0
chembl-webresource-client>=0.10.8
python-dateutil>=2.8.2
httpx>=0.24.0
defusedxml>=0.7.1
psycopg2-binary>=2.9.0
bedrock-agentcore-starter-toolkit
pubmedmcp

Overwriting requirements.txt


## 4. 환경 변수 설정

In [5]:
# AWS 세션 설정
boto_session = Session()
region = boto_session.region_name
account_id = boto3.client('sts').get_caller_identity()['Account']

# 환경 변수 설정 (실제 값으로 변경 필요)
import os

# 내부 데이터베이스 설정
os.environ['DB_HOST'] = 'YOUR_RDS_ENDPOINT'  # RDS 엔드포인트로 변경
os.environ['DB_PORT'] = '5432'
os.environ['DB_NAME'] = 'agentdb'
os.environ['DB_USER'] = 'dbadmin'
os.environ['DB_PASSWORD'] = 'postgres'

# 단백질 설계 설정
STACK_NAME = 'protein-design-stack'
os.environ['WORKFLOW_ID'] = '8600464'  # CloudFormation 출력에서 가져오기
os.environ['WORKFLOW_ROLE_ARN'] = f'arn:aws:iam::{account_id}:role/{STACK_NAME}-WorkflowExecutionRole'
os.environ['S3_BUCKET'] = f'{STACK_NAME}-{account_id}-{region}'

# Knowledge Base 설정
os.environ['KNOWLEDGE_BASE_ID'] = 'YOUR_KB_ID'  # Knowledge Base ID로 변경

print(f"Region: {region}")
print(f"Account ID: {account_id}")
print(f"환경 변수 설정 완료")

Region: us-east-1
Account ID: 797157869634
환경 변수 설정 완료


## 5. AgentCore Runtime 구성 및 배포

In [6]:
%%writefile requirements.txt
strands-agents>=0.5.0
strands-agents-tools>=0.1.0
boto3>=1.34.0
mcp>=0.1.0
arxiv>=2.1.0
chembl-webresource-client>=0.10.8
python-dateutil>=2.8.2
python-dotenv>=1.0.0
httpx>=0.24.0
defusedxml>=0.7.1
psycopg2-binary>=2.9.0
bedrock-agentcore-starter-toolkit
pubmedmcp

Overwriting requirements.txt


In [7]:
# AWS 세션 설정
boto_session = Session()
region = boto_session.region_name

# Runtime 인스턴스 생성
agentcore_runtime = Runtime()
agent_name = "integrated_lifescience_research_agent"

# 환경 변수를 .env 파일로 저장
env_content = f"""DB_HOST={os.environ['DB_HOST']}
DB_PORT={os.environ['DB_PORT']}
DB_NAME={os.environ['DB_NAME']}
DB_USER={os.environ['DB_USER']}
DB_PASSWORD={os.environ['DB_PASSWORD']}
WORKFLOW_ID={os.environ['WORKFLOW_ID']}
WORKFLOW_ROLE_ARN={os.environ['WORKFLOW_ROLE_ARN']}
S3_BUCKET={os.environ['S3_BUCKET']}
KNOWLEDGE_BASE_ID={os.environ.get('KNOWLEDGE_BASE_ID', '')}"""

with open('.env', 'w') as f:
    f.write(env_content)
print("✅ 환경 변수 파일 생성 완료")

response = agentcore_runtime.configure(
    entrypoint="integrated_research_agent.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name
)
print(response)

Entrypoint parsed: file=/home/sagemaker-user/strands-agents-for-life-science/notebook/test/integrated_research_agent.py, bedrock_agentcore_name=integrated_research_agent
INFO:bedrock_agentcore_starter_toolkit.utils.runtime.entrypoint:Entrypoint parsed: file=/home/sagemaker-user/strands-agents-for-life-science/notebook/test/integrated_research_agent.py, bedrock_agentcore_name=integrated_research_agent
Configuring BedrockAgentCore agent: integrated_lifescience_research_agent
INFO:bedrock_agentcore_starter_toolkit.operations.runtime.configure:Configuring BedrockAgentCore agent: integrated_lifescience_research_agent


✅ 환경 변수 파일 생성 완료


⚠️  ℹ️  No container engine found (Docker/Finch/Podman not installed)
✅ Default deployment uses CodeBuild (no container engine needed)
💡 Run 'agentcore launch' for cloud-based building and deployment
💡 For local builds, install Docker, Finch, or Podman

⚠️  [WARNING] Platform mismatch: Current system is 'linux/amd64' but Bedrock AgentCore requires 'linux/arm64'.
For deployment options and workarounds, see: 
https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/getting-started-custom.html

Generated Dockerfile: /home/sagemaker-user/strands-agents-for-life-science/notebook/test/Dockerfile
INFO:bedrock_agentcore_starter_toolkit.operations.runtime.configure:Generated Dockerfile: /home/sagemaker-user/strands-agents-for-life-science/notebook/test/Dockerfile
Generated .dockerignore: /home/sagemaker-user/strands-agents-for-life-science/notebook/test/.dockerignore
INFO:bedrock_agentcore_starter_toolkit.operations.runtime.configure:Generated .dockerignore: /home/sagemaker-user/strands-agents-for-life-science/notebook/test/.dockerignore
Keeping 'integrated_lifescience_research_agent' as default agent
INFO:bedrock_agentcore_starter_toolkit.utils.runtime.config:Keeping 'integrated_lifescience_research_agent' as default agent
Bedrock AgentCore configured: /home/sagemaker-user/strands-agents-for-life-science/notebook/test/.bedrock_agentcore.yaml
INFO:bedrock_agentcore_starter_toolkit.notebook.runtime.bedrock_agentcore:Bedrock AgentCore configured: /home/sagemaker-user/strands-agents-f

config_path=PosixPath('/home/sagemaker-user/strands-agents-for-life-science/notebook/test/.bedrock_agentcore.yaml') dockerfile_path=PosixPath('/home/sagemaker-user/strands-agents-for-life-science/notebook/test/Dockerfile') dockerignore_path=PosixPath('/home/sagemaker-user/strands-agents-for-life-science/notebook/test/.dockerignore') runtime='None' region='us-east-1' account_id='797157869634' execution_role=None ecr_repository=None auto_create_ecr=True


## 6. Agent 배포

In [8]:
# AgentCore Runtime에 배포
print("🚀 통합 Agent를 AgentCore Runtime에 배포 중...")
launch_result = agentcore_runtime.launch()

print(f"\n✅ 배포 완료!")
print(f"Agent ARN: {launch_result.agent_arn}")
print(f"ECR URI: {launch_result.ecr_uri}")

🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
INFO:bedrock_agentcore_starter_toolkit.notebook.runtime.bedrock_agentcore:🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
   • Build ARM64 containers in the cloud with CodeBuild
INFO:bedrock_agentcore_starter_toolkit.notebook.runtime.bedrock_agentcore:   • Build ARM64 containers in the cloud with CodeBuild
   • No local Docker required
INFO:bedrock_agentcore_starter_toolkit.notebook.runtime.bedrock_agentcore:   • No local Docker required
💡 Available deployment modes:
INFO:bedrock_agentcore_starter_toolkit.notebook.runtime.bedrock_agentcore:💡 Available deployment modes:
   • runtime.launch()                           → CodeBuild (current)
INFO:bedrock_agentcore_starter_toolkit.notebook.runtime.bedrock_agentcore:   • runtime.launch()                           → CodeBuild (current)
   • runtime.launch(local=True)                 → Local development
INFO:bedrock_agentcore_starter_toolkit.notebook.runtime.bedrock_agentc

🚀 통합 Agent를 AgentCore Runtime에 배포 중...
✅ Reusing existing ECR repository: 797157869634.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-integrated_lifescience_research_agent


✅ Reusing existing execution role: arn:aws:iam::797157869634:role/AmazonBedrockAgentCoreSDKRuntime-us-east-1-547438ac38
INFO:bedrock_agentcore_starter_toolkit.operations.runtime.launch:✅ Reusing existing execution role: arn:aws:iam::797157869634:role/AmazonBedrockAgentCoreSDKRuntime-us-east-1-547438ac38
✅ Execution role available: arn:aws:iam::797157869634:role/AmazonBedrockAgentCoreSDKRuntime-us-east-1-547438ac38
INFO:bedrock_agentcore_starter_toolkit.operations.runtime.launch:✅ Execution role available: arn:aws:iam::797157869634:role/AmazonBedrockAgentCoreSDKRuntime-us-east-1-547438ac38
Preparing CodeBuild project and uploading source...
INFO:bedrock_agentcore_starter_toolkit.operations.runtime.launch:Preparing CodeBuild project and uploading source...
Getting or creating CodeBuild execution role for agent: integrated_lifescience_research_agent
INFO:bedrock_agentcore_starter_toolkit.services.codebuild:Getting or creating CodeBuild execution role for agent: integrated_lifescience_rese


✅ 배포 완료!
Agent ARN: arn:aws:bedrock-agentcore:us-east-1:797157869634:runtime/integrated_lifescience_research_agent-m5RclbEjj5
ECR URI: 797157869634.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-integrated_lifescience_research_agent


## 7. 배포 상태 확인

In [9]:
import time

# 배포 상태 확인
print("⏳ Agent 상태 확인 중...")
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']

end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(f"상태: {status}")

print(f"\n✅ 최종 상태: {status}")

⏳ Agent 상태 확인 중...


Retrieved Bedrock AgentCore status for: integrated_lifescience_research_agent
INFO:bedrock_agentcore_starter_toolkit.notebook.runtime.bedrock_agentcore:Retrieved Bedrock AgentCore status for: integrated_lifescience_research_agent



✅ 최종 상태: READY


## 8. 통합 Agent 테스트

다양한 시나리오로 통합 Agent를 테스트합니다.

### 8.1 외부 데이터베이스 통합 테스트

In [ ]:
# 테스트 1: 문헌 검색 통합
query1 = """COVID-19 백신과 관련된 최신 연구를 조사해주세요. 
Arxiv와 PubMed에서 관련 논문을 찾고, 주요 내용을 요약해주세요."""

print("📚 테스트 1: 문헌 검색 통합")
print(f"질문: {query1}")
print("=" * 60)

response1 = agentcore_runtime.invoke({"prompt": query1})
if 'response' in response1:
    print(f"응답:\n{response1['response'][0]}")
print("=" * 60)

In [ ]:
# 테스트 2: 화합물 정보
query2 = """Aspirin의 화학적 특성과 생물학적 활성을 조사해주세요.
ChEMBL 데이터베이스에서 IC50 값과 관련 연구도 포함해주세요."""

print("🧪 테스트 2: 화합물 정보 검색")
print(f"질문: {query2}")
print("=" * 60)

response2 = agentcore_runtime.invoke({"prompt": query2})
if 'response' in response2:
    print(f"응답:\n{response2['response'][0]}")
print("=" * 60)

### 8.2 내부 데이터 분석 테스트

In [10]:
# 테스트 2: 화합물 정보
query3 = """데이터베이스에 어떤 테이블들이 있는지 알려주세요."""

print("🧪 테스트 3: 사내 데이터베이스 에이전트 테스트")
print(f"질문: {query3}")
print("=" * 60)

response3 = agentcore_runtime.invoke({"prompt": query3})
if 'response' in response3:
    print(f"응답:\n{response3['response'][0]}")
print("=" * 60)

🧪 테스트 3: 사내 데이터베이스 에이전트 테스트
질문: 데이터베이스에 어떤 테이블들이 있는지 알려주세요.
응답:
죄송합니다. 현재 데이터베이스 연결에 문제가 있어 스키마 정보를 가져올 수 없습니다. 데이터베이스 서버에 연결할 수 없는 상태입니다. 

다음과 같은 오류가 발생했습니다:
"서버 "localhost"(127.0.0.1), 포트 5432에 대한 연결 실패: 연결이 거부됨. 서버가 해당 호스트에서 실행 중이고 TCP/IP 연결을 수락하고 있습니까?"

이 문제는 다음과 같은 이유로 발생할 수 있습니다:
1. 데이터베이스 서버가 현재 실행되지 않고 있음
2. 데이터베이스가 다른 호스트나 포트에서 실행 중임
3. 방화벽이나 네트워크 설정으로 연결이 차단됨

데이터베이스 관리자에게 연락하여 이 문제를 해결하고 나중에 다시 시도해 보시길 권장드립니다. 혹시 필요하신 다른 정보나 도움이 있으시면 말씀해 주세요.


### 8.3 단백질 설계 테스트

In [ ]:
# 테스트 4: 단백질 최적화
test_sequence = "EVQLVETGGGLVQPGGSLRLSCAASGFTLNSYGISWVRQAPGKGPEWVS"
query4 = f"""다음 항체 서열을 최적화해주세요: {test_sequence}
안정성과 결합 친화력을 향상시키는 방향으로 최적화해주세요."""

print("🧬 테스트 4: 단백질 설계")
print(f"질문: {query4}")
print("=" * 60)

response4 = agentcore_runtime.invoke({"prompt": query4})
if 'response' in response4:
    print(f"응답:\n{response4['response'][0]}")
print("=" * 60)

### 8.5 복합 연구 질문 테스트

In [ ]:
# 테스트 5: 복합 연구 질문
query5 = """알츠하이머병 치료를 위한 새로운 접근법에 대해 종합적으로 조사해주세요:
1. 최신 연구 논문 (Arxiv, PubMed)
2. 현재 임상시험 중인 약물 (ChEMBL)
3. 우리 데이터베이스의 관련 환자 데이터
4. 타우 단백질 응집 억제를 위한 펩타이드 설계 가능성
"""

print("🔬 테스트 5: 복합 연구 질문")
print(f"질문: {query5}")
print("=" * 60)

response5 = agentcore_runtime.invoke({"prompt": query5})
if 'response' in response5:
    print(f"응답:\n{response5['response'][0]}")
print("=" * 60)

## 9. 성능 모니터링

In [ ]:
# CloudWatch 메트릭 확인
cloudwatch = boto3.client('cloudwatch', region_name=region)

from datetime import datetime, timedelta

end_time = datetime.utcnow()
start_time = end_time - timedelta(hours=1)

try:
    metrics = cloudwatch.get_metric_statistics(
        Namespace='AWS/BedrockAgentCore',
        MetricName='InvocationCount',
        Dimensions=[
            {'Name': 'AgentName', 'Value': agent_name}
        ],
        StartTime=start_time,
        EndTime=end_time,
        Period=300,
        Statistics=['Sum']
    )
    
    print("📊 최근 1시간 호출 통계:")
    for datapoint in metrics.get('Datapoints', []):
        print(f"  시간: {datapoint['Timestamp']}, 호출 수: {datapoint['Sum']}")
        
    # 평균 응답 시간
    latency_metrics = cloudwatch.get_metric_statistics(
        Namespace='AWS/BedrockAgentCore',
        MetricName='InvocationLatency',
        Dimensions=[
            {'Name': 'AgentName', 'Value': agent_name}
        ],
        StartTime=start_time,
        EndTime=end_time,
        Period=300,
        Statistics=['Average']
    )
    
    print("\n⏱️ 평균 응답 시간:")
    for datapoint in latency_metrics.get('Datapoints', []):
        print(f"  시간: {datapoint['Timestamp']}, 평균: {datapoint['Average']:.2f}ms")
        
except Exception as e:
    print(f"메트릭 조회 실패: {e}")

## 10. 정리 (Optional)

In [ ]:
# Agent Runtime 삭제 (필요시에만 실행)
# 주의: 이 셀을 실행하면 배포된 Agent가 삭제됩니다!

# cleanup = input("정말로 Agent를 삭제하시겠습니까? (yes/no): ")
# if cleanup.lower() == 'yes':
#     agentcore_control_client = boto3.client(
#         'bedrock-agentcore-control',
#         region_name=region
#     )
#     
#     ecr_client = boto3.client('ecr', region_name=region)
#     
#     try:
#         # Runtime 삭제
#         runtime_delete_response = agentcore_control_client.delete_agent_runtime(
#             agentRuntimeId=launch_result.agent_id
#         )
#         
#         # ECR 리포지토리 삭제
#         ecr_response = ecr_client.delete_repository(
#             repositoryName=launch_result.ecr_uri.split('/')[1],
#             force=True
#         )
#         
#         print("✅ Agent 정리 완료")
#     except Exception as e:
#         print(f"정리 중 오류: {e}")

## 요약

이 노트북에서는 모든 생명과학 연구 도구를 통합한 종합 Agent를 Amazon Bedrock AgentCore Runtime에 성공적으로 배포했습니다.

### 통합된 기능:
- ✅ **외부 데이터베이스**: Arxiv, ChEMBL, PubMed 문헌 및 화합물 검색
- ✅ **내부 데이터베이스**: PostgreSQL 임상/유전체 데이터 분석
- ✅ **단백질 설계**: AWS HealthOmics 워크플로우 기반 최적화
- ✅ **Knowledge Base**: 조직 내부 지식 검색 및 활용

### 주요 성과:
- 모든 도구를 하나의 Agent로 통합
- 서버리스 환경에서 확장 가능한 배포
- 실시간 연구 질문 처리
- 다양한 생명과학 연구 시나리오 지원

### 활용 시나리오:
1. **신약 개발**: 타겟 발굴부터 리드 최적화까지
2. **정밀 의료**: 환자 맞춤형 치료 전략 수립
3. **바이오마커 발굴**: 진단 및 예후 마커 개발
4. **단백질 공학**: 치료용 항체 및 효소 설계
5. **임상 연구**: 데이터 기반 의사결정 지원

### 다음 단계:
1. 추가 데이터 소스 통합 (KEGG, UniProt 등)
2. 실시간 스트리밍 응답 구현
3. 멀티모달 지원 (이미지, 구조 데이터)
4. 사용자 인증 및 접근 제어
5. 연구 결과 자동 보고서 생성